# EP1 - Cálculo do Conjunto de Mandelbrot em Paralelo com Pthreads e OpenMP

25 de Maio de 2020

## Membros do grupo

| Nome | NUSP |
|------|------|
| Carolina Marques | 10737101 |
| Daniela Favero | 10277443 |
| Miguel Ostrowski | 10723610 |
| Raphael Ribeiro | 10281601 |

## Pacotes Julia

Instalando os pacotes necessários que estão listados no arquivo `Project.toml`:

In [1]:
] up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Fetching: [=====>                                   ]  12.4 %>                             ]  26.6 %Fetching: [==============>                          ]  34.4 %>                    ]  48.5 %==========================>              ]  62.8 %63.4 %Fetching: [============================>            ]  69.4 %Fetching: [==================================>      ]  82.9 %> ]  97.0 % Resolving package versions...
 Installed FriBidi_jll ──────── v1.0.5+3
 Installed libvorbis_jll ────── v1.3.6+4
 Installed libass_jll ───────── v0.14.0+2
 Installed DataStructures ───── v0.17.15
 Installed StatsPlots ───────── v0.14.6
 Installed StaticArrays ─────── v0.12.3
 Installed DataFrames ───────── v0.21.0
 Installed RecipesPipeline ──── v0.1.9
 Installed x265_jll ─────────── v3.0.0+1
 Installed CategoricalArrays ── v0.8

  Building GR ───→ `~/.julia/packages/GR/cRdXQ/deps/build.log`
  Building Plots → `~/.julia/packages/Plots/zOV0T/deps/build.log`


Verificando o status dos pacotes:

In [2]:
] st

    Status `~/Documents/julia/concorrente-e-paralela/EP1/doc/Project.toml`
  [336ed68f] CSV v0.6.2
  [a93c6f00] DataFrames v0.21.0
  [31c24e10] Distributions v0.23.2
  [7073ff75] IJulia v1.21.2
  [8314cec4] PGFPlotsX v1.2.6
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6


## Compilando

Compilando e executando os códigos C pelo *modo shell*: 

**!!!!!!! FIXME: Precisamos de um makefile, executar com os parâmetros certos, mostrar que está funcionando, etc**
OBS.: Pensei em botar umas imagenzinhas dos mandelbrot aqui, o que vcs acham?

In [1]:
; make mandelbrot_seq.c

make: Nothing to be done for 'mandelbrot_seq.c'.


In [2]:
; ./mandelbrot_seq

usage: ./mandelbrot_seq c_x_min c_x_max c_y_min c_y_max image_size
examples with image_size = 11500:
    Full Picture:         ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500
    Seahorse Valley:      ./mandelbrot_seq -0.8 -0.7 0.05 0.15 11500
    Elephant Valley:      ./mandelbrot_seq 0.175 0.375 -0.1 0.1 11500
    Triple Spiral Valley: ./mandelbrot_seq -0.188 -0.012 0.554 0.754 11500


## Experimentos

### Funções úteis

**!!!!!!! FIXME: Essas funções serão modificadas conforme os parâmetros desse EP, por enquanto essa função só foi assaltada do MiniEP3**

A função abaixo recebe parâmetros `size`, com tamanho da amostra, `f`, com a id da função a estimar, e `threads`, com o número de threads do programa paralelo. A função executa o programa `monte_carlo` com os parâmetros dados e devolve um `DataFrame` com os resultados.

In [ ]:
using DataFrames, Query, StatsPlots, Statistics

function measure_monte_carlo(size, f, threads)
    results = parse.(Float64,
        split(chomp(read(`./monte_carlo $size $f $threads`, String)), ", "))
        
    return DataFrame(size = size,
        f = f,
        threads = threads,
        estimate = results[1],
        duration = results[2])
end

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`, e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [ ]:
function run_experiments(size, f, threads, repetitions)
    run(`make`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        estimate = Float64[],
        duration = Float64[])  
    
    for t in threads
        for r in 1:repetitions
            append!(results,
                measure_monte_carlo(size, f, t))    
        end
    end
    
    return results
end

A função `parse_results` recebe um `DataFrame` de resultados, produzido pela função `run_experiments`, e um parâmetro `target_integral`, com o valor da integral a estimar. A função devolve um `DataFrame` com a média e o intervalo de confiança da média a 95% das estimativas e dos tempos de execução, agrupados por número de threads.

In [ ]:
function parse_results(results, target_integral)
    parsed_results = results |>
                    @groupby(_.threads) |>
                    @map({threads = key(_),
                          mean_estimate = mean(_.estimate),
                          ci_estimate = 1.96 * std(_.estimate),
                          mean_duration = mean(_.duration),
                          ci_duration = 1.96 * std(_.duration),
                          target = target_integral}) |>
                    DataFrame
    
    return parsed_results
end

### Gerando CSV

A função `save_csv_results`recebe um `DataFrame` e um nome de arquivo, e escreve o `DataFrame` em disco, no formato `.csv`, com o nome passado no argumento.

In [ ]:
using CSV

function save_csv_results(data_frame, file)
    CSV.write(file, data_frame)
end

save_csv_results(results, "results.csv")

A função `read_csv_results` recebe um nome de arquivo e lê o arquivo correspondente, devolvendo um `DataFrame`.

In [ ]:
using CSV

function read_csv_results(file)
    return CSV.read(file)
end

results = read_csv_results("results.csv")

**!!!!!!! FIXME: Na real tem que fazer essa parte inteira ainda né kkk Geração de gráficos e tudo mais**

- Medições de tempo de execução para diferentes tamanhos de entrada e diferentes números de threads

- Pelo menos 10 medições para cada experimento, usando média e intervalo de confiança das medições nos gráficos. Cada experimento deverá ser repetido nas 4 regiões especificadas de Mandelbrot (ver tabela 1)

- Não esquece de gerar o csv

- Análise das porções não paralelizáveis do código sequencial: I/O e alocação de memória

## Discussão

**!!!!!!! FIXME: responder tudo isso aqui**

- Como e por que as três versões do programa se comportam com a variação:
    - Do tamanho da entrada?
    - Das regiões do Conjunto de Mandelbrot?
    - Do número de threads?

- Qual o impacto das operações de I/O e alocação de memória no tempo de execução?

- Mais perguntas interessantes...